In [115]:
import pandas as pd
import numpy as np 
from random import shuffle

from scikit_alarm_framework.alarm_generator import set_point 

import time

import concurrent.futures
import multiprocessing

In [2]:
proc_path = 'proc_i3e.csv'
proc_df = pd.read_csv(proc_path, index_col='TOUT', usecols=['TOUT']+["XMEAS%02d" % x for x in  [1,2,3,6,7,8,9,21]])
proc_df.index = pd.to_datetime(proc_df.index, unit='s')
proc_df.head()

,XMEAS01,XMEAS02,XMEAS03,XMEAS06,XMEAS07,XMEAS08,XMEAS09,XMEAS21
TOUT,,,,,,,,
2019-12-10 16:54:17.616330147,0.271033,3649.739415,4451.320791,47.559754,2798.975799,64.995825,122.898796,102.480028
2019-12-10 16:54:53.616330147,0.270093,3663.351750,4428.605105,47.917222,2799.059838,65.254940,122.883398,102.484819
2019-12-10 16:55:29.616330147,0.271255,3656.090868,4429.093949,47.750928,2799.345791,64.930642,122.909663,102.499473
2019-12-10 16:56:05.616330147,0.269950,3664.897833,4439.129238,47.305023,2799.251300,65.188788,122.909316,102.465085
2019-12-10 16:56:41.616330147,0.270947,3616.540495,4446.078748,47.813862,2798.967112,65.686033,122.887484,102.463271


In [12]:
threshold_path = 'dist6_3horas_sig/threshold.csv'

threshold_df = pd.read_csv(threshold_path, index_col='proc_var')
threshold_df = threshold_df.loc[[1,2,3,6,7,8,9,21]]

thresh_low = threshold_df[threshold_df['type'] == 'LOW']['limit'].values
thresh_high = threshold_df[threshold_df['type'] == 'HIGH']['limit'].values

high_alarms = set_point.generate(proc_df, thresh_high)
low_alarms = set_point.generate(proc_df, thresh_low)

new_high_names = ['x' + str(i) + '_high' for i, col in enumerate(proc_df.columns)]
new_low_names = ['x' + str(i) + '_low' for i, col in enumerate(proc_df.columns)]

rename_high = {key:value for key, value in zip(high_alarms.columns,new_high_names)}
rename_low = {key:value for key, value in zip(low_alarms.columns,new_low_names)}

high_alarms = high_alarms.rename(rename_high, axis='columns')
low_alarms = low_alarms.rename(rename_low, axis='columns')

alarms_df = pd.concat([high_alarms, low_alarms], axis=1)

In [114]:
a = np.arange(0,100)
b = a.copy()
def sum_test(a, b):
    c = []
    for el, el1 in zip(a,b):
        c.append(el + el1)
    return c

def fact(n):
    if n == 1 or n == 0:
        return 1
    else:
        return fact(n-1)*n

def find(target, v):
    for el in v:
        if el == target:
            return el

def test():
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        init = time.process_time()
        result = executor.submit(sum_test, a, b)
        end = time.process_time()
        print(result.result())
    end = time.process_time()
    print(end - init)

    
def test1():
    init = time.process_time()
    pool = multiprocessing.Pool(4)
    pool.apply(sum_test,(a, b))
    end = time.process_time()
    
    print(end - init)



In [111]:
def test2():
    init = time.process_time()
    pool = multiprocessing.Pool(2)
    r = pool.apply(fact,(200,))
    end = time.process_time()
    
    print(end - init)
    return (r)


In [150]:
def test3():
    init = time.process_time()
    pool = multiprocessing.Pool(2)
    r = pool.apply(find,(3,vec))
    end = time.process_time()
    
    print(end - init)
    return (r)

In [86]:
init = time.process_time()
r = sum_test(a, b)
end = time.process_time()

print(end - init)
print(r)



0.0002869939999996518
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198]


In [88]:
test()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198]
0.06263740700000042


In [137]:
test1()
    

0.07427142400000264


In [148]:
vec = np.arange(0,10000000)
shuffle(vec)


init = time.process_time()
find(3, vec)
end = time.process_time()

print(end - init)



0.9397251400000073


In [151]:
test3()

0.2989377739999952


3

In [108]:
init = time.process_time()
r = fact(200)
end = time.process_time()

print(end - init)
print(r)


0.0004175149999987582
788657867364790503552363213932185062295135977687173263294742533244359449963403342920304284011984623904177212138919638830257642790242637105061926624952829931113462857270763317237396988943922445621451664240254033291864131227428294853277524242407573903240321257405579568660226031904170324062351700858796178922222789623703897374720000000000000000000000000000000000000000000000000
